Vraagjes aan de docenten:
- How to calculate the block time for aircrafts that fly routes overnight? Which part of the flight is assigned to which day
- How to make sure the most optimal route replaces the previous most optimal route?


In [2]:
# Group: 28
# Authors: Thieme Brandwagt and Sam Buisman
#=========================================
# Assignment 2 - Aircraft Routing Problem 
#=========================================

# Import necessary packages
from openpyxl import *
from time import *
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# profit, n, aircraft, i, j = s # Attributes for state


### STEP 1: Load the data

In [58]:
# Load airport data into a numpy array
file_path_airports = "DATA/AirportData.xlsx"
airports_df = pd.read_excel(file_path_airports, usecols ="B:U")
airports = airports_df.to_numpy()
print(airports[0][1])
# Create list for all the airports (columns)
cities = airports_df.columns
print(cities[0])


# Load fleet data into a numpy array
file_path_fleet = "DATA/FleetType.xlsx"
fleet_df = pd.read_excel(file_path_fleet)
fleet_df = fleet_df.set_index("Aircraft Type")
fleet = fleet_df.to_numpy()
print(fleet[0][0])   # speed, capacity, tat, range, runway, lease cost, fixed cost, time cost, fuel cost, amount

# Create list for all the aircraft types (columns)
aircraft_types = fleet_df.columns
print(aircraft_types[0])


# Load demand data into a numpy array
file_path_demand = "DATA/Group28.xlsx"
demand_df = pd.read_excel(file_path_demand, usecols="B:AG", skiprows=4)
demand = demand_df.to_numpy()
print(demand[0][0])

# Create a triple dimensional array for demand data (origin x destination x time)
demand_matrix = np.zeros((len(cities), len(cities), demand.shape[1] - 2))
# Populate matrix with all demand from all intervals
for rows in range(len(cities)):
    for columns in range(len(cities)):
        for time_steps in range(int(120/4)):  # 120 hours devided by 4 hour time steps
            demand_matrix[rows][columns][time_steps] = demand[rows * 20 + columns][time_steps + 2]
print(demand_matrix[19][18][6])


CDG
London
800.0
Type 1: Small freighter
LHR
11396.314893372413


In [38]:
time_intervals = ["00:00-04:00", "04:00-08:00", "08:00-12:00", 
                  "12:00-16:00", "16:00-20:00", "20:00-00:00"]

In [39]:
tijd = [0,1,2,3,4,5]
tijd[-1]

5

### STEP 2: Define the functions

In [40]:
# Adding the previous two bins to the currect bin to get total demand
def flow(i, j, t):
    return demand_matrix[i,j,t] + 0.2 * (demand_matrix[i,j,t-1]+demand_matrix[i,j,t-2])

print(flow(0,1,25)) # LHR to CDG for bin 8

51127.0017609541


In [41]:
def distance(i, j, R_E=6371):
    if i != j:
        lat_i, long_i = airports[1][i], airports[2][i]
        lat_j, long_j = airports[1][j], airports[2][j]
        delta_lat = lat_j - lat_i
        delta_long = long_j - long_i
        root = math.sin(math.radians(delta_lat)/2)**2 + math.cos(math.radians(lat_i)) \
            * math.cos(math.radians(lat_j)) * math.sin(math.radians(delta_long)/2)**2
        return R_E * 2 * math.asin(math.sqrt(root)) 
    else:
        return 0

# E.v.t. nog stoppen in een df'pje later, zodat ie niet naar zichzelf gaat verwijzen

In [42]:
time_step = 6
total_time = 120 * 60
# print(stages)
# print(stages[5])

def flight_duration(i,j,k):
    return distance(i, j, R_E=6371) / fleet[0][k]

time_stages = np.arange(0, total_time, time_step)
time_bins = np.arange(30)

divider = len(time_stages)/ len(time_bins) # Divide by 40 to get the right bin associated with the stage (round down)

print(len(time_stages))
print(divider)

1200
40.0


In [43]:
# Calculate the cost for each flight leg
def cost_flight_leg(i, j, k):
    if i != j:
        tc = fleet[7][k] * (distance(i, j, R_E=6371) / fleet[0][k])     # formula for the time costs
        fc = ((fleet[8][k] * 1.42) / 1.5) * distance(i, j, R_E=6371)    # formula for the fuel costs
        foc = fleet[6][k]                                               # formula for the fixed operating costs
        return tc + fc + foc

print(cost_flight_leg(0,1,0))

# Calculate the profit for each flight leg
def profit(i, j, k, n):
    satisfied_demand = min(flow(i,j,math.floor(n / divider)), fleet[1][k])
    if i != j:
        revenue = 0.26 * distance(i,j) * satisfied_demand / 1000
        costs = cost_flight_leg(i,j,k)
        return revenue - costs
print(flow(0,1,26))
print(profit(0, 1, 0, 1050))
print(fleet[1][0])

# Calculate the flight duration for each flight leg
# ONDERSTAANDE TWEE FUNCTIES KUNNEN LATER EVT GEMERGED WORDEN
def flight_duration(i,j,k):
    return distance(i, j, R_E=6371) / fleet[0][k] * 60 + 15 + 15 # Round up to the nearest 6-fold
print(flight_duration(1,2,1)) # From Paris to Amsterdam

def get_stages_in_flight(i,j,k):
    return math.ceil((flight_duration(i,j,k)) / 6)

print(get_stages_in_flight(1,2,1))

2385.302572558103
8403.575400678272
-1626.7664906330197
23000.0
58.113158915127194
10


In [44]:
# # DEZE CEL KAN LATER MISSCHIEN WEG, MAAR WE ZOUDEN DE STATUS KUNNEN BEPALEN DOOR DUS ALTIJD EEN I EN J MEE TE GEVEN AAN EEN VLIEGTUIG, DUS I NAAR I ALS HET VLIEGTUIG STILSTAAT OP DE HUB EN J NAAR J OP EEN SPOKE, I NAAR J ALS IE ONDERWEG IS
# state = ['grounded_hub', 
#           'grounded_spoke', 
#           'airborne_to_hub', 
#           'airborne_to_spoke'] 

# # Determine current state (what is the current status of the plane?)
# def get_current_state(i, j):
#     if i == j:  # grounded
#         return state[0] if i == 3 else state[1] # grounded at hub or spoke
#     else:  
#         return state[2] if j == 3 else state[3] # airborne to hub or to spoke

# get_current_state(4,4)

In [45]:
# Create a list of all possible decisions a plane can take at airport i at arrival stage n
def get_candidates_decisions(i,k,n):    
    candidates = []
    hub = 3 # Define Frankfurt as a hub
    if i == hub:                 
        for j in range(len(cities)):                                                        
            if i == j:
                candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,n)})                                                         # grounded at hub
            else:
                departure_stage = n-get_stages_in_flight(i,j,k)
                candidates.append({"From":cities[i], "to":cities[j],"profit":profit(i,j,k,departure_stage), "departure stage":departure_stage})  # flight to spoke
        
    else:
        candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,n)})   
        departure_stage = n-get_stages_in_flight(i,hub,k)                                                       # grounded at spoke
        candidates.append({"From":cities[i], "to":cities[hub], "profit":profit(i,hub,k,departure_stage), "departure stage":departure_stage})     # flight to hub
    return candidates

get_candidates_decisions(3,0,1000)

[{'From': 'Frankfurt',
  'to': 'London',
  'profit': -3733.623089404598,
  'departure stage': 986},
 {'From': 'Frankfurt',
  'to': 'Paris',
  'profit': -179.95064787450292,
  'departure stage': 989},
 {'From': 'Frankfurt',
  'to': 'Amsterdam',
  'profit': -284.599331892211,
  'departure stage': 990},
 {'Grounding at': 'Frankfurt', 'profit': None},
 {'From': 'Frankfurt',
  'to': 'Madrid',
  'profit': -1381.8944017992671,
  'departure stage': 977},
 {'From': 'Frankfurt',
  'to': 'Barcelona',
  'profit': -1649.5338991357003,
  'departure stage': 981},
 {'From': 'Frankfurt',
  'to': 'Munich',
  'profit': -1889.0091382163673,
  'departure stage': 991},
 {'From': 'Frankfurt',
  'to': 'Rome',
  'profit': -4549.823401285375,
  'departure stage': 983},
 {'From': 'Frankfurt',
  'to': 'Dublin',
  'profit': -2627.1559130161363,
  'departure stage': 981},
 {'From': 'Frankfurt',
  'to': 'Stockholm',
  'profit': 802.6841347241616,
  'departure stage': 979},
 {'From': 'Frankfurt',
  'to': 'Lisbon',
  

In [46]:
# Pick the list item with the greatest profit
def optimal_decision(i,k,s):
    return max(
        get_candidates_decisions(i,k,s),
        key = lambda x:x['profit'] if x['profit'] is not None else float('-inf')
        )

print(f"The most profitable decision is: {optimal_decision(3,0,-100)}")

The most profitable decision is: {'From': 'Frankfurt', 'to': 'Palermo', 'profit': 147.6206929163218, 'departure stage': -123}


In [47]:
print(get_stages_in_flight(3,6,0))

9


### STEP 3: Building the algorithm

## NOTES

In [48]:
ending_state = [[-1200], [3], [0], [0], [None]]   # stage, airport, summed profit, hours flown, route flown
len(ending_state[1])
print(ending_state[1][0])

3


In [49]:
new_states = [[-1200, 3, 0, 0, [None, "Test"]],
                [-1199, 3, 0, 0, [None, "Test"]],
                [-1198, 5, -200, 0, [None, "Test", "Pest", "Vest"]]
                ]
print(len(new_states))
print(new_states[0])
print(new_states[0][1])
print(new_states[1][1])
print(new_states[0][4][1])
print(new_states[2][4][3])
print(len(new_states[2][4]))

3
[-1200, 3, 0, 0, [None, 'Test']]
3
3
Test
Vest
4


In [50]:
new_states = [[-1200, 3, 0, 0, [None, "Test"]]]
print(new_states[0][1])

3


In [51]:
ending_state = [-1200, 3, 0, 0, None]   # stage, airport, maximum profit, hours flown (day), route flown

def test_state(ending_state):
    print(ending_state[0])
    print(ending_state[1])
    print(ending_state[2])
    print(ending_state[3])
    print(ending_state[4])

test_state(ending_state)

fleet[3][0]

test = []
test.append(1)
test.append([2,2])
test.append(5)
print(test)

-1200
3
0
0
None
[1, [2, 2], 5]


In [52]:
# Create a list of all possible decisions a plane can take at airport i at arrival stage n
def get_candidates_decisions(i,k,n):    
    candidates = []
    hub = 3 # Define Frankfurt as a hub
    if i == hub:                 
        for j in range(len(cities)):                                                        
            if i == j:
                candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,n)})                                                         # grounded at hub
            else:
                departure_stage = n-get_stages_in_flight(i,j,k)
                candidates.append({"From":cities[i], "to":cities[j],"profit":profit(i,j,k,departure_stage), "departure stage":departure_stage})  # flight to spoke
        
    else:
        candidates.append({"Grounding at":cities[i], "profit":profit(i,i,k,n)})   
        departure_stage = n-get_stages_in_flight(i,hub,k)                                                       # grounded at spoke
        candidates.append({"From":cities[i], "to":cities[hub], "profit":profit(i,hub,k,departure_stage), "departure stage":departure_stage})     # flight to hub
    return candidates

def flow(i, j, t):
    return demand_matrix[i,j,t] + 0.2 * (demand_matrix[i,j,t-1]+demand_matrix[i,j,t-2])

def distance(i, j):
    if i != j:
        lat_i, long_i = airports[1][i], airports[2][i]
        lat_j, long_j = airports[1][j], airports[2][j]
        delta_lat = lat_j - lat_i
        delta_long = long_j - long_i
        root = math.sin(math.radians(delta_lat)/2)**2 + math.cos(math.radians(lat_i)) \
            * math.cos(math.radians(lat_j)) * math.sin(math.radians(delta_long)/2)**2
        return 6371 * 2 * math.asin(math.sqrt(root)) 
    else:
        return 0
    
# Calculate the cost for each flight leg
def cost_flight_leg(i, j, k):
    if i != j:
        tc = fleet[7][k] * (distance(i, j) / fleet[0][k])     # formula for the time costs
        fc = ((fleet[8][k] * 1.42) / 1.5) * distance(i, j)    # formula for the fuel costs
        foc = fleet[6][k]                                               # formula for the fixed operating costs
        return tc + fc + foc

print(cost_flight_leg(0,1,0))

# Calculate the profit for each flight leg
def profit(i, j, k, n):
    satisfied_demand = min(flow(i,j,math.floor(n / divider)), fleet[1][k])
    if i != j:
        revenue = 0.26 * distance(i,j) * satisfied_demand / 1000
        costs = cost_flight_leg(i,j,k)
        return revenue - costs
print(flow(0,1,26))
print(profit(0, 1, 0, 1050))
print(fleet[1][0])

# Calculate the flight duration for each flight leg
# ONDERSTAANDE TWEE FUNCTIES KUNNEN LATER EVT GEMERGED WORDEN
def flight_block_time(i,j,k):
    return distance(i, j) / fleet[0][k] * 60 + 15 + 15   # not rounded to full stages
def flight_duration(i,j,k):
    return math.ceil((flight_block_time(i,j,k) + fleet[2][0]) / 6)   # rounded up to full stages

print(flight_duration(1,2,1)) # From Paris to Amsterdam

2385.302572558103
8403.575400678272
-1626.7664906330197
23000.0
25


In [67]:
new_states = []
if flight_details:
	print([1199-flight_details[0][2], 2, 100 + flight_details[0][6], 200 + flight_details[0][5], flight_details[0]])
else:
	print("flight_details is empty")

[1170, 2, -3734.209530801696, 279.10727249396774, [0, 3, 29, 1170, 1199, 79.10727249396774, -3834.209530801696, 0.0]]


In [97]:
# Gather all the details for the flight between i (origin) and j (destination (=previous_state)) with airplane k
def get_flight_details(previous_states, s, i, k):
    global flight_details
    flight_details = []
    n = previous_states[s][0]              # stage
    j = previous_states[s][1]              # destination
    flight_time = flight_duration(i,j,k)
    block_time = flight_block_time(i,j,k)
    departure_time = n - flight_time
    arrival_time = n
    flight_profit = profit(i,j,k,n)
    satisfied_demand = min(flow(i,j,math.floor(n / divider)), fleet[1][k])
    flight_details.append([i, j, flight_time, departure_time, arrival_time, block_time, flight_profit, satisfied_demand])

get_flight_details([[1040, 3, 0, 0, None]], 0, 0, 0)
print(flight_details)
print(flight_details[0][2])

[[0, 3, 29, 1011, 1040, 79.10727249396774, -1845.240439951918, 11683.423956593228]]
29


In [94]:
# Create a list of all possible routes a plane had before arriving (or staying) at airport j (=previous_state) on stage n
def get_new_states(previous_states, k):
    global new_states
    new_states = []
    for s in range(len(previous_states)):   
        hub = 3     # define Frankfurt as a hub
        n, j, sum_profit, hours_flown, route_flown  = previous_states[s][0], previous_states[s][1], previous_states[s][2], previous_states[s][3], previous_states[s][4]
        if j == hub:    # hub is destination                
            for i in range(len(cities)):                                                        
                if i == j:  # grounded at hub
                    new_states.append([n-1, hub, sum_profit, hours_flown, route_flown])                                                         
                else:       # flying from spoke to hub
                    get_flight_details(previous_states, s, i, k)
                    if distance(i,j) <= fleet[3][k] and airports[3][i] >= fleet[4][k] and airports[3][hub] >= fleet[4][k] \
                        and n - flight_details[s][2] >= 0:
                        new_states.append([n-flight_details[s][2], i, sum_profit + flight_details[s][6], hours_flown + flight_details[s][5], flight_details[s]])
        else:           # spoke is destination
            new_states.append([n-1, j, sum_profit, hours_flown, route_flown])   # grounded at spoke
            get_flight_details(previous_states, s, hub, k)                      # flying from hub to spoke
            if distance(hub,j) <= fleet[3][k] and airports[3][j] >= fleet[4][k] and airports[3][hub] >= fleet[4][k] \
                        and n - flight_details[s][2] >= 0:
                new_states.append([n-flight_details[s][2], hub, sum_profit + flight_details[s][6], hours_flown + flight_details[s][5], flight_details[s]])
    return new_states

get_new_states([[1199, 3, 0, 0, None]], 0)

[[1170,
  0,
  -3834.209530801696,
  79.10727249396774,
  [0, 3, 29, 1170, 1199, 79.10727249396774, -3834.209530801696, 0.0]],
 [1173,
  1,
  -2865.0009602161936,
  63.675380171509495,
  [1, 3, 26, 1173, 1199, 63.675380171509495, -2865.0009602161936, 0.0]],
 [1174,
  2,
  -2476.732705270283,
  57.49331153902264,
  [2, 3, 25, 1174, 1199, 57.49331153902264, -2476.732705270283, 0.0]],
 [1198, 3, 0, 0, None],
 [1161,
  4,
  -7448.450715875428,
  136.65379291088695,
  [4, 3, 38, 1161, 1199, 136.65379291088695, -7448.450715875428, 0.0]],
 [1165,
  5,
  -5901.365379151117,
  112.02085521868209,
  [5, 3, 34, 1165, 1199, 112.02085521868209, -5901.365379151117, 0.0]],
 [1175,
  6,
  -2158.7615024812303,
  52.43052370160296,
  [6, 3, 24, 1175, 1199, 52.43052370160296, -2158.7615024812303, 0.0]],
 [1167,
  7,
  -5263.190183819763,
  101.85972871185822,
  [7, 3, 32, 1167, 1199, 101.85972871185822, -5263.190183819763, 0.0]],
 [1165,
  8,
  -5869.723704877819,
  111.51705147085427,
  [8, 3, 34, 1165,

## ACTUAL CODE

In [77]:
ending_state = [1199, 3, 0, 0, None]       # stage, airport, maximum profit, hours flown, route flown
new_states = []                             # stage, airport, maximum profit, hours flown, route flown
best_route_state = [1199, 3, 0, 0, None]   # stage, airport, maximum profit, hours flown, route flown
flight = [3, 1, 10, 5, 15, 4, 500, 20000]   # origin, destination, flight duration, departure, arrival, block time, profit, satisfied demand
available_aircraft_types = [0,1,1]          # amount of aircrafts per type (first type, then amount)

In [72]:
k = fleet[9]    # amount of aircraft types
print(k)

[2. 2. 1.]


In [113]:
# Create a list of all possible routes a plane had before arriving (or staying) at airport j (=previous_state) on stage n
def get_new_states(previous_states, k):
    global new_states
    new_states = []
    for s in range(len(previous_states)):   
        hub = 3     # define Frankfurt as a hub
        n, j, sum_profit, hours_flown, route_flown  = previous_states[s][0], previous_states[s][1], previous_states[s][2], previous_states[s][3], previous_states[s][4]
        if j == hub:    # hub is destination                
            for i in range(len(cities)):                                                        
                if i == j:  # grounded at hub
                    new_states.append([n-1, hub, sum_profit, hours_flown, route_flown])                                                         
                else:       # flying from spoke to hub
                    get_flight_details(previous_states, s, i, k)
                    print(flight_details)    
                    if distance(i,j) <= fleet[3][k] and airports[3][i] >= fleet[4][k] and airports[3][hub] >= fleet[4][k] \
                        and n - flight_details[s][2] >= 0:
                        new_states.append([n-flight_details[s][2], i, sum_profit + flight_details[s][6], hours_flown + flight_details[s][5], flight_details[s]])
        else:           # spoke is destination
            new_states.append([n-1, j, sum_profit, hours_flown, route_flown])   # grounded at spoke
            get_flight_details(previous_states, s, hub, k)
            print(flight_details)                 # flying from hub to spoke
            if n - flight_details[s][2] >= 0 and distance(hub,j) <= fleet[3][k] \
                and airports[3][j] >= fleet[4][k] and airports[3][hub] >= fleet[4][k]:
                    new_states.append([n-flight_details[s][2], hub, sum_profit + flight_details[s][6], hours_flown + flight_details[s][5], flight_details[s]])
    return new_states

# print(get_new_states([[1199, 3, 0, 0, None]], 0))

# Gather all the details for the flight between i (origin) and j (destination (=previous_state)) with airplane k
def get_flight_details(previous_states, s, i, k):
    global flight_details
    flight_details = []
    n = previous_states[s][0]              # stage
    j = previous_states[s][1]              # destination
    flight_time = flight_duration(i,j,k)
    block_time = flight_block_time(i,j,k)
    departure_time = n - flight_time
    arrival_time = n
    flight_profit = profit(i,j,k,n)
    satisfied_demand = min(flow(i,j,math.floor(n / divider)), fleet[1][k])
    flight_details.append([i, j, flight_time, departure_time, arrival_time, block_time, flight_profit, satisfied_demand])

# get_flight_details([[1040, 3, 0, 0, None]], 0, 0, 0)
# print(flight_details)
# print(flight_details[0][2])

#=========================================================================================================

# Set initial values
iterations = 0
ending_state = [1199, 3, 0, 0, None]   # stage, airport, maximum profit, hours flown, route flown
available_aircraft_types = fleet[9]    # amount of aircrafts for each type at the start

# for k in range(len(available_aircraft_types)):
#     # Check if there are any aircrafts available of type k
#     if available_aircraft_types[k] >= 1:
# First iteration starts with the ending state
k=0
while iterations < 10:
    if iterations == 0:
        get_new_states([ending_state], k)
        iterations += 1
        print(new_states)
    else:
        previous_states = new_states
        new_states = []
        get_new_states(previous_states, k)
        for s in range(len(new_states)):
            if new_states[s][2] > best_route_state[2]:
                best_route_state = new_states[s]
        iterations += 1
        print(new_states)
#         global optimal_route
#         optimal_route = best_route_state[2].max()

# print(iterations)
# print(optimal_route)


[[0, 3, 29, 1170, 1199, 79.10727249396774, -3834.209530801696, 0.0]]
[[1, 3, 26, 1173, 1199, 63.675380171509495, -2865.0009602161936, 0.0]]
[[2, 3, 25, 1174, 1199, 57.49331153902264, -2476.732705270283, 0.0]]
[[4, 3, 38, 1161, 1199, 136.65379291088695, -7448.450715875428, 0.0]]
[[5, 3, 34, 1165, 1199, 112.02085521868209, -5901.365379151117, 0.0]]
[[6, 3, 24, 1175, 1199, 52.43052370160296, -2158.7615024812303, 0.0]]
[[7, 3, 32, 1167, 1199, 101.85972871185822, -5263.190183819763, 0.0]]
[[8, 3, 34, 1165, 1199, 111.51705147085427, -5869.723704877819, 0.0]]
[[9, 3, 36, 1163, 1199, 121.7240381523955, -6510.779173960173, 0.0]]
[[10, 3, 44, 1155, 1199, 170.53852952114102, -9576.600423536105, 0.0]]
[[11, 3, 26, 1173, 1199, 62.376196837266086, -2783.4050291405174, 0.0]]
[[12, 3, 40, 1159, 1199, 145.34542443824324, -7994.333462635222, 0.0]]
[[13, 3, 32, 1167, 1199, 97.21809569632043, -4971.669843593903, 0.0]]
[[14, 3, 33, 1166, 1199, 107.46438492732987, -5615.193731130357, 0.0]]
[[15, 3, 39, 1160

IndexError: list index out of range

In [ ]:
# Create a list of all possible routes a plane had before arriving (or staying) at airport j (=previous_state) on stage n
def get_new_states(previous_states, k):
    global new_states
    new_states = []
    for s in range(len(previous_states)):   
        hub = 3     # define Frankfurt as a hub
        n, j, sum_profit, hours_flown, route_flown  = previous_states[s][0], previous_states[s][1], previous_states[s][2], previous_states[s][3], previous_states[s][4]
        if j == hub:    # hub is destination                
            for i in range(len(cities)):                                                        
                if i == j:  # grounded at hub
                    new_states.append([n-1, hub, sum_profit, hours_flown, route_flown])                                                         
                else:       # flying from spoke to hub
                    get_flight_details(previous_states, s, i, k)
                    if distance(i,j) <= fleet[3][k] and airports[3][i] >= fleet[4][k] and airports[3][hub] >= fleet[4][k] \
                        and n - flight_details[s][2] >= 0:
                        new_states.append([n-flight_details[s][2], i, sum_profit + flight_details[s][6], hours_flown + flight_details[s][5], flight_details[s]])
        else:           # spoke is destination
            new_states.append([n-1, j, sum_profit, hours_flown, route_flown])   # grounded at spoke
            get_flight_details(previous_states, s, hub, k)                      # flying from hub to spoke
            if n - flight_details[s][2] >= 0 and distance(hub,j) <= fleet[3][k] \
                and airports[3][j] >= fleet[4][k] and airports[3][hub] >= fleet[4][k]:
                new_states.append([n-flight_details[s][2], hub, sum_profit + flight_details[s][6], hours_flown + flight_details[s][5], flight_details[s]])
    return new_states

print(get_new_states([[1199, 3, 0, 0, None]], 0))

# Gather all the details for the flight between i (origin) and j (destination (=previous_state)) with airplane k
def get_flight_details(previous_states, s, i, k):
    global flight_details
    flight_details = []
    n = previous_states[s][0]              # stage
    j = previous_states[s][1]              # destination
    flight_time = flight_duration(i,j,k)
    block_time = flight_block_time(i,j,k)
    departure_time = n - flight_time
    arrival_time = n
    flight_profit = profit(i,j,k,n)
    satisfied_demand = min(flow(i,j,math.floor(n / divider)), fleet[1][k])
    flight_details.append([i, j, flight_time, departure_time, arrival_time, block_time, flight_profit, satisfied_demand])

get_flight_details([[1040, 3, 0, 0, None]], 0, 0, 0)
print(flight_details)
print(flight_details[0][2])

#=========================================================================================================

# Set initial values
iterations = 0
ending_state = [1199, 3, 0, 0, None]   # stage, airport, maximum profit, hours flown, route flown
available_aircraft_types = fleet[9]    # amount of aircrafts for each type at the start

# for k in range(len(available_aircraft_types)):
#     # Check if there are any aircrafts available of type k
#     if available_aircraft_types[k] >= 1:
# First iteration starts with the ending state
k=0
while iterations < 10:
    if iterations == 0:
        get_new_states([ending_state], k)
        iterations += 1
        print(new_states)
    else:
        previous_states = new_states
        new_states = []
        get_new_states(previous_states, k)
        for s in range(len(new_states)):
            if new_states[s][2] > best_route_state[2]:
                best_route_state = new_states[s]
        iterations += 1

print(iterations)
#         global optimal_route
#         optimal_route = best_route_state[2].max()

# print(optimal_route)

### Helper Functions

In [80]:
def calculate_revenue(route, flow):                                                                           # Bij ons binnenin def profit(i,j,k,t)
    """Calculate revenue for a given route and cargo flow."""
    distance = FLIGHT_DATA[route]["distance"]
    yield_per_rtk = 0.26  # Revenue per RTK
    return yield_per_rtk * distance * flow

def calculate_cost(route, aircraft_type):                                                                     # Bij ons binnenin def profit(i,j,k,t)
    """Calculate operating costs for a given route and aircraft."""
    flight_data = FLIGHT_DATA[route]
    time_cost = (flight_data["distance"] / aircraft_type["speed"]) * aircraft_type["time_cost"]
    fuel_cost = (flight_data["distance"] ** 1.5) * aircraft_type["fuel_cost"]
    fixed_cost = aircraft_type["fixed_cost"]
    return time_cost + fuel_cost + fixed_cost

def transition(state, decision):                                                                              
    """Compute the next state and associated cost/reward."""
    t, airport, aircraft = state
    route = decision["route"]
    if route is None:  # No flight decision
        return (t + TIME_STEP, airport, aircraft), 0
    next_airport = route["destination"]
    flight_time = route["flight_time"]
    if t + flight_time > TOTAL_TIME:
        return None, 0  # Invalid transition
    revenue = calculate_revenue(route, decision["cargo"])
    cost = calculate_cost(route, aircraft)
    return (t + flight_time, next_airport, aircraft), revenue - cost

def dp_solve():
    """Dynamic programming solver."""
    for t in reversed(time_stages):
        for airport in AIRPORTS:
            for aircraft in AIRCRAFT_TYPES:
                state = (t, airport, aircraft)
                dp_table[state] = -np.inf  # Initialize with negative infinity
                best_decision = None
                # Evaluate decisions
                for decision in get_possible_decisions(state):                                          # Dit deel is bij ons nu gedekt door def optimal_decision(i,k,t)
                    next_state, reward = transition(state, decision)
                    if next_state is not None:
                        value = reward + dp_table.get(next_state, 0)
                        if value > dp_table[state]:
                            dp_table[state] = value
                            best_decision = decision
                # Store best decision for traceback
                dp_table[state] = {"value": dp_table[state], "decision": best_decision}

def get_possible_decisions(state):                                                                      # Bij ons def get_candidate_decisions
    """Generate possible decisions for a given state."""
    t, airport, aircraft = state
    decisions = []
    # Add all possible flights
    for route in FLIGHT_DATA:
        if route["origin"] == airport:
            # Check constraints (range, runway, etc.)
            decisions.append({"route": route, "cargo": min(route["demand"], MAX_CAPACITY[aircraft])})   # In onze functie is nog geen cargo meegenomen
    # Add idle (no flight) decision
    decisions.append({"route": None, "cargo": 0})
    return decisions

def traceback():
    """Traceback to reconstruct the optimal solution."""
    schedule = []
    state = (0, HUB, AIRCRAFT_TYPES[0])  # Starting state
    while state in dp_table:
        best_decision = dp_table[state]["decision"]
        if best_decision["route"] is not None:
            schedule.append({"state": state, "decision": best_decision})
        next_state, _ = transition(state, best_decision)
        state = next_state
    return schedule

# Main Execution
def main():
    # Load data from Excel
    # AIRCRAFT_TYPES = pd.read_excel("FleetType.xlsx")
    # AIRPORTS = pd.read_excel("AirportData.xlsx")
    # DEMAND = pd.read_excel("GroupX.xlsx")
    # Preprocess FLIGHT_DATA and DEMAND
    # TODO

    # Solve DP
    dp_solve()

    # Traceback to get the optimal solution
    optimal_schedule = traceback()

    # Print or save results
    print("Optimal Schedule:")
    for entry in optimal_schedule:
        print(entry)

if __name__ == "__main__":
    main()

NameError: name 'AIRPORTS' is not defined

Loop over all possible itineraries

In [ ]:
for t in trips:
    while trip == feasible:
            continue
    else:
        break


In [74]:
def get_flight_details(previous_states, i, k):
    global flight_details
    flight_details = []
    n = previous_states[0]              # stage
    j = previous_states[1]              # destination
    flight_time = flight_duration(i,j,k)
    block_time = flight_block_time(i,j,k)
    departure_time = n - flight_time
    arrival_time = n
    flight_profit = profit(i,j,k,n)
    satisfied_demand = min(flow(i,j,math.floor(n / divider)), fleet[1][k])
    flight_details.append([i, j, flight_time, departure_time, arrival_time, block_time, flight_profit, satisfied_demand])

get_flight_details([1040, 3, 0, 0, None], 0, 0)
print(flight_details)

[[0, 3, 29, 1011, 1040, 79.10727249396774, -1845.240439951918, 11683.423956593228]]
